<a href="https://colab.research.google.com/github/yas-cgs/estudo-projetos-java/blob/main/Teste_planilha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
from google.colab import files

# Enviar o arquivo de credenciais do seu computador
uploaded = files.upload()


Saving adroit-bonsai-464923-k4-8f73e52bff75.json to adroit-bonsai-464923-k4-8f73e52bff75.json


In [59]:
import os
os.rename("adroit-bonsai-464923-k4-8f73e52bff75.json", "credenciais.json")
#os.rename("adroit-bonsai-464923-k4-9fb123da1b2f.json", "credenciais.json")


In [ ]:
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("credenciais.json", scope)
client = gspread.authorize(creds)


In [ ]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials

# Configurações de autenticação
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("credenciais.json", scope)
client = gspread.authorize(creds)

# Abrir a planilha pelo nome ou URL
planilha = client.open("Consulta_ATIVIDADESCTS")

# Selecionar a aba pelo nome
worksheet = planilha.worksheet("DB_TABULAR")

valores = worksheet.get_all_values()

# Pula as duas primeiras linhas (index 0 e 1), pois o cabeçalho real está na linha 2 (index 1)
dados_reais = valores[2:]

colunas = [
    "ANO",
    "Data",
    "Pesquisador(a)",
    "Coautor(a)",
    "Tipo de atividade",
    "Título",
    "Portal de Publicação/Organizador",
    "Âmbito",
    "Link",
    "Projeto",
    "Coluna 11"
]

df = pd.DataFrame(dados_reais, columns=colunas)

# Agora converta "Data" para datetime
df["Data"] = pd.to_datetime(df["Data"], dayfirst=True, errors="coerce")

print(df.head())


    ANO       Data   Pesquisador(a) Coautor(a)  \
0  2024 2024-08-20  ⁨Erica Bakonyi⁩              
1  2024 2024-06-20     Yasmin Curzi              
2  2024 2024-06-26     Yasmin Curzi              
3  2024 2024-07-01     Yasmin Curzi              
4  2024 2024-07-15     Yasmin Curzi              

                                Tipo de atividade  \
0                          Participação em evento   
1    Autoria de artigo publicado na imprensa/blog   
2                  Entrevista em tv/rádio/podcast   
3  Entrevista/aspas/participação jornal/blog/site   
4  Entrevista/aspas/participação jornal/blog/site   

                                              Título  \
0      Encontro preparatório da Cúpula Social do G20   
1  Desafios para a Representatividade Política de...   
2  Menina de 12 anos sofre com gordofobia após ví...   
3  Campeões de emendas a projeto sobre IA no Sena...   
4  Com demissões e sem ‘PL das Fake News’, 'big t...   

  Portal de Publicação/Organizador    Âmbit

In [ ]:
import datetime

# 1. Filtragem por data
inicio = datetime.datetime(2025, 1, 1)
fim = datetime.datetime(2025, 6, 16)

df_filtrado = df[
    (df["Data"] >= inicio) &
    (df["Data"] <= fim) &
    (df["Título"].notna()) &
    (df["Tipo de atividade"].notna())
].copy()

# 2. Funções para formatar nomes conforme FGV
def formatar_nome(nome):
    sufixos = ["Filho", "Júnior", "Neto"]
    particulas = ["de", "da", "do", "das", "dos"]

    if not isinstance(nome, str) or nome.strip() == "":
        return ""

    partes = nome.strip().split()
    if not partes:
        return ""

    sufixo = ""
    if partes[-1] in sufixos:
        sufixo = " " + partes.pop()

    # Busca o último sobrenome ignorando partículas
    for i in range(len(partes)-1, -1, -1):
        if partes[i].lower() not in particulas:
            sobrenome = partes[i].upper() + sufixo
            restante = partes[:i] + partes[i+1:]
            break
    else:
        # Caso todos sejam partículas, pega o último mesmo assim
        sobrenome = partes[-1].upper()
        restante = partes[:-1]

    nome_formatado = sobrenome + ", " + " ".join(restante)
    return nome_formatado.strip()

def formatar_autores(lista_nomes):
    nomes_unicos = []
    for n in lista_nomes:
        fn = formatar_nome(n)
        if fn and fn not in nomes_unicos:
            nomes_unicos.append(fn)
    if len(nomes_unicos) > 3:
        return f"{nomes_unicos[0]} et al."
    else:
        return "; ".join(nomes_unicos)

# 3. Agrupar por Tipo de atividade + Título e consolidar autores
referencias = []

agrupado = df_filtrado.groupby(["Tipo de atividade", "Título"], dropna=False)

for (tipo, titulo), grupo in agrupado:
    autores_set = set()
    for _, row in grupo.iterrows():
        if pd.notna(row["Pesquisador(a)"]) and row["Pesquisador(a)"].strip():
            autores_set.add(row["Pesquisador(a)"].strip())
        if pd.notna(row["Coautor(a)"]) and row["Coautor(a)"].strip():
            autores_set.add(row["Coautor(a)"].strip())

    autores_formatados = formatar_autores(list(autores_set))
    primeira_linha = grupo.iloc[0]

    data_fmt = primeira_linha["Data"].strftime("%d %b. %Y").lower() if pd.notna(primeira_linha["Data"]) else "[S.d.]"
    portal = primeira_linha.get("Portal de Publicação/Organizador") or "SN"
    link = primeira_linha.get("Link") or "SN"

    referencia = f"[{tipo}] {autores_formatados}. {titulo}. {portal}, {data_fmt}. Disponível em: {link}."

    referencias.append({
        "Tipo de atividade": tipo,
        "Título": titulo,
        "Referência formatada": referencia
    })

# 4. Criar DataFrame final ordenado
df_referencias = pd.DataFrame(referencias).sort_values("Tipo de atividade").reset_index(drop=True)

# Mostrar as primeiras referências
print(df_referencias.head())

# 5. (Opcional) Exportar para CSV
df_referencias.to_csv("referencias_formatadas.csv", index=False)

   Tipo de atividade                                             Título  \
0      Amicus curiae  Contribuição ao Tema Repetitivo nº 1.315 no ST...   
1  Audiência Pública  Concorrência nos ecossistemas digitais de disp...   
2               Aula     Inteligência Artificial e Judiciário, 24 horas   
3               Aula  International Trade and International Arbitration   
4               Aula  Perspectivas sobre Proteção de Dados e Privaci...   

                                Referência formatada  
0  [ Amicus curiae] MUCELIN, Guilherme. Contribui...  
1  [Audiência Pública] ZINGALES, Nicolo. Concorrê...  
2  [Aula] NUNES, José Luiz. Inteligência Artifici...  
3  [Aula] ZINGALES, Nicolo. International Trade a...  
4  [Aula] GOMES, Luca Belli; Rodrigo Dias de Pinh...  


In [ ]:
def formatar_referencia_sem_tipo(row, autores_formatados):
    titulo = row["Título"]
    titulo_it = f"*{titulo}*"
    data_str = abreviar_data(row["Data"])
    portal = row.get("Portal de Publicação/Organizador") or "SN"
    link = row.get("Link") or "SN"

    ref = f"{autores_formatados}. {titulo_it}. {portal}, {data_str}. Disponível em: {link}."
    return ref

# Usar este formato na geração da lista

referencias_dict = {}

agrupado = df_filtrado.groupby(["Tipo de atividade", "Título"], dropna=False)

for (tipo, titulo), grupo in agrupado:
    autores_set = set()
    for _, row in grupo.iterrows():
        if pd.notna(row["Pesquisador(a)"]) and row["Pesquisador(a)"].strip():
            autores_set.add(row["Pesquisador(a)"].strip())
        if pd.notna(row["Coautor(a)"]) and row["Coautor(a)"].strip():
            autores_set.add(row["Coautor(a)"].strip())
    autores_formatados = formatar_autores(list(autores_set))
    primeira_linha = grupo.iloc[0]
    referencia = formatar_referencia_sem_tipo(primeira_linha, autores_formatados)
    if tipo not in referencias_dict:
        referencias_dict[tipo] = []
    primeiro_autor = autores_formatados.split(";")[0] if ";" in autores_formatados else autores_formatados.split(" et al.")[0]
    referencias_dict[tipo].append({
        "referencia": referencia,
        "primeiro_autor": primeiro_autor,
        "titulo": titulo
    })

for tipo in referencias_dict:
    referencias_dict[tipo].sort(key=lambda x: (x["primeiro_autor"], x["titulo"]))

saida_texto = ""
for tipo in sorted(referencias_dict.keys()):
    saida_texto += f"\n{tipo}\n"
    for item in referencias_dict[tipo]:
        saida_texto += f"{item['referencia']}\n"

print(saida_texto)




 Amicus curiae
MUCELIN, Guilherme. *Contribuição ao Tema Repetitivo nº 1.315 no STJ – Em Defesa dos Direitos dos Consumidores no Ambiente Digital*. Amicus Curiae, 14 mai. 2025. Disponível em: https://www.linkedin.com/posts/guilherme-mucelin-phd-686a2b17a_it-was-a-privilege-to-contribute-to-the-amicus-activity-7326338035002793984-Vxfp?utm_source=share&utm_medium=member_desktop&rcm=ACoAAFVsGQ8BFFOTJVS6GCWiTJtPemux_0Kq1cY.

Audiência Pública
ZINGALES, Nicolo. *Concorrência nos ecossistemas digitais de dispositivos móveis*. CADE, 19 fev. 2025. Disponível em: https://www.gov.br/cade/pt-br/assuntos/noticias/cade-realizara-audiencia-publica-para-debater-concorrencia-nos-ecossistemas-digitais-de-dispositivos-moveis.

Aula
GOMES, Luca Belli; Rodrigo Dias de Pinho. *Perspectivas sobre Proteção de Dados e Privacidade no Brasil*. FGV Direito Rio, 8 mai. 2025. Disponível em: https://direitorio.fgv.br/eventos/aula-aberta-perspectivas-sobre-protecao-de-dados-e-privacidade-no-brasil.
NUNES, José Luiz

In [ ]:
!pip install python-docx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 4.7 MB/s eta 0:00:00


In [ ]:
from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn

# Criar documento Word
doc = Document()

# Define estilo padrão: Times New Roman 12
style = doc.styles['Normal']
font = style.font
font.name = 'Times New Roman'
font.size = Pt(12)

# Função para aplicar formatação ao título (negrito) e à referência (com título em itálico)
def adicionar_referencia_paragrafo(paragrafo, referencia):
    partes = referencia.split("*")
    for i, parte in enumerate(partes):
        run = paragrafo.add_run(parte)
        if i % 2 == 1:  # parte ímpar = título (em itálico)
            run.italic = True
        run.font.name = 'Times New Roman'
        run.font.size = Pt(12)

# Criar doc por tipo
for tipo in sorted(referencias_dict.keys()):
    # Adicionar título da seção (tipo de atividade)
    p_titulo = doc.add_paragraph()
    run_titulo = p_titulo.add_run(tipo)
    run_titulo.bold = True
    run_titulo.font.name = 'Times New Roman'
    run_titulo.font.size = Pt(12)

    # Adicionar referências dentro da seção
    for item in referencias_dict[tipo]:
        p_ref = doc.add_paragraph(style='Normal')
        adicionar_referencia_paragrafo(p_ref, item['referencia'])

# Salvar o documento
doc.save("referencias_formatadas.docx")


In [ ]:
from google.colab import files
files.download("referencias_formatadas.docx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>